<a href="https://colab.research.google.com/github/AshmitB05/cuda-parallel-programming-ece408/blob/main/ECE408_Assignments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp1rua9its".


In [ ]:
# Detect selected GPU and its NVIDA architecture:
import subprocess
gpu_info = subprocess.getoutput("nvidia-smi --query-gpu=name,compute_cap --format=csv,noheader,nounits")
if "not found" in gpu_info.lower(): raise RuntimeError("Error: No GPU found. Please select a GPU runtime environment.")
gpu_name, compute_cap = map(str.strip, gpu_info.split(','))
gpu_arch = f"sm_{compute_cap.replace('.', '')}"

print(f"{'GPU Name':<15}: {gpu_name}")
print(f"{'Architecture':<15}: {gpu_arch}")

GPU Name       : Tesla T4
Architecture   : sm_75


In [ ]:
%%cuda -c "--gpu-architecture $gpu_arch"
#include <stdio.h>

int main(){
int devcount;
cudaGetDeviceCount(&devcount);
printf("Number of devices: %d\n", devcount);
cudaDeviceProp prop;
for(int i=0;i<devcount;i++){
    cudaGetDeviceProperties(&prop,i);
    printf("Device %d: %s\n",i,prop.name);
    printf("Device %d: compute capablity %d.%d\n",i,prop.major,prop.minor);
    printf("Device %d: Warp size %d\n",i,prop.warpSize);
    printf("Device %d: Max threads per block %d x %d y %d z\n",i,prop.maxThreadsDim[0],prop.maxThreadsDim[1],prop.maxThreadsDim[2]);
    printf("Device %d: Max blocks per grid %d x %d y %d z\n",i,prop.maxGridSize[0],prop.maxGridSize[1],prop.maxGridSize[2]);
    printf("Device %d: Global Memory size %d kb \n",i,prop.totalGlobalMem/1024);
    printf("Device %d: Shared Memory per block %d kb \n",i,prop.sharedMemPerBlock/1024);
    printf("Device %d: Constant Memory %d kb \n",i,prop.totalConstMem/1024);
    printf("Device %d: Max threads per multiprocessor %d \n",i,prop.maxThreadsPerMultiProcessor);
    printf("Device %d: Registers per block %d\n",i,prop.regsPerBlock);
    printf("clock rate %d\n",prop.clockRate);
    return 0;



}}

Number of devices: 1
Device 0: Tesla T4
Device 0: compute capablity 7.5
Device 0: Warp size 32
Device 0: Max threads per block 1024 x 1024 y 64 z
Device 0: Max blocks per grid 2147483647 x 65535 y 65535 z
Device 0: Global Memory size 15457344 kb 
Device 0: Shared Memory per block 48 kb 
Device 0: Constant Memory 64 kb 
Device 0: Max threads per multiprocessor 1024 
Device 0: Registers per block 65536
clock rate 1590000



In [ ]:
#MP0 Sum of an array

%%cuda -c "--gpu-architecture $gpu_arch"
#include <stdio.h>
#define N 1000
__global__ void kernel(int *a,int *b,int *c,int n){
    int i=blockIdx.x*blockDim.x+threadIdx.x;
    if(i<n){
        c[i]=a[i]+b[i];
    }
}
int main(){
    int a[N],b[N],c[N];
    int *da,*db,*dc;
    cudaMalloc((void**)&da,N*sizeof(int));
    cudaMalloc((void**)&db,N*sizeof(int));
    cudaMalloc((void**)&dc,N*sizeof(int));
    for(int i=0;i<N;i++){
        a[i]=i;
        b[i]=i;
    }
    cudaEvent_t start,end;
    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);
    cudaMemcpy(da,a,N*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(db,b,N*sizeof(int),cudaMemcpyHostToDevice);
    dim3 grid(ceil(N/256.0),1,1);
    dim3 block(256,1,1);
    kernel<<<grid,block>>>(da,db,dc,N);
    cudaMemcpy(c,dc,N*sizeof(int),cudaMemcpyDeviceToHost);
    cudaEventRecord(end);
    cudaEventSynchronize(end);
    float time=0;
    cudaEventElapsedTime(&time,start,end);
    printf("GPU Time taken: %f ms\n",time);
    cudaDeviceSynchronize();
    cudaEventDestroy(start);
    cudaEventDestroy(end);
    int d[N];
    cudaEvent_t start1,end1;
    cudaEventCreate(&start1);
    cudaEventCreate(&end1);
    cudaEventRecord(start1);

    for(int i=0;i<N;i++){
        d[i]=a[i]+b[i];
    }
    cudaEventRecord(end1);
    cudaEventSynchronize(end1);
    float time1=0;
    cudaEventElapsedTime(&time1,start1,end1);
    printf("CPU Time taken: %f ms\n",time1);
    int count=0;
    for(int i=0;i<N;i++){
        if(c[i]!=d[i]){
            count++;
            printf("%d %d , %d\n", i,c[i],d[i]);
        }
    }
    if(count==0){
        printf("Correct\n");
     }
    else{
        printf("failure\n");
    }
    cudaFree(da);
    cudaFree(db);
    cudaFree(dc);

    return 0;



}

GPU Time taken: 1.166688 ms
CPU Time taken: 0.005056 ms
Correct



In [ ]:
#MP1 mat mul


%%cuda -c "--gpu-architecture $gpu_arch"
#define l 64
#define r 66
#define width 65
#include <stdio.h>
__global__ void kernel(int *a,int *b,int *c){
    int row=blockIdx.y*blockDim.y+threadIdx.y;
    int col=blockIdx.x*blockDim.x+threadIdx.x;

    if(row<l && col<r){
        int p=0;
        for(int i=0;i<width;i++){
            p+=a[row*width+i]*b[i*r+col];
        }
        c[row*r+col]=p;
    }

}
int main(){
    int a[l*width],b[width*r],c[l*r],d[l*r];
    for(int i=0;i<l;i++){
        for(int j=0;j<width;j++){
            a[i*width+j]=i*width+j;
        }
    }
    for(int i=0;i<width;i++){
        for(int j=0;j<r;j++){
            b[i*r+j]=i*r+j;
        }

  }
    cudaEvent_t start,end;
    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);
    for(int i=0;i<l;i++){
        for(int j=0;j<r;j++){
            for(int k=0;k<width;k++){
                d[i*r+j]+=a[i*width+k]*b[k*r+j];
        }

    }}
    cudaEventRecord(end);
    cudaEventSynchronize(end);
    float time=0;
    cudaEventElapsedTime(&time,start,end);
    printf("CPU Time taken: %f ms\n",time);
    int *da,*db,*dc;
    cudaMalloc((void**)&da,l*width*sizeof(int));
    cudaMalloc((void**)&db,width*r*sizeof(int));
    cudaMalloc((void**)&dc,l*r*sizeof(int));
    cudaEvent_t start1,end1;
    cudaEventCreate(&start1);
    cudaEventCreate(&end1);
    cudaEventRecord(start1);
    cudaMemcpy(da,a,l*width*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(db,b,width*r*sizeof(int),cudaMemcpyHostToDevice);
    dim3 grid(ceil(r/16.0),ceil(l/16.0),1);
    dim3 block(16,16,1);
    kernel<<<grid,block>>>(da,db,dc);
    cudaMemcpy(c,dc,l*r*sizeof(int),cudaMemcpyDeviceToHost);
    cudaEventRecord(end1);
    cudaEventSynchronize(end1);
    float time1=0;
    cudaEventElapsedTime(&time1,start1,end1);
    printf("GPU Time taken: %f ms\n",time1);
    int count=0;
    for(int i=0;i<l;i++){
        for(int j=0;j<r;j++){
            if(c[i*r+j]!=d[i*r+j]){
                count++;
                printf("%d %d %d , %d\n", i,j,c[i*r+j],d[i*r+j]);
            }}}
    if(count==0){
        printf("Correct\n");
     }
    else{
        printf("failure\n");}
    cudaFree(da);
    cudaFree(db);
    cudaFree(dc);
    return 0;

}


CPU Time taken: 1.716224 ms
GPU Time taken: 3.714336 ms
Correct



In [ ]:
#MP2 tiled mat mul

%%cuda -c "--gpu-architecture $gpu_arch"
#define l 4
#define r 4
#define width 3
#define tile 2
#include <stdio.h>
__global__ void kernel(int *a,int *b,int *c){
    int row=blockIdx.y*blockDim.y+threadIdx.y;
    int col=blockIdx.x*blockDim.x+threadIdx.x;
    __shared__ int Mds[tile][tile];
    __shared__ int Nds[tile][tile];
    int p=0;
    for(int i=0;i<((width/tile)+1);i++){
        if(row<l && (i*tile+threadIdx.x)<width){
            Mds[threadIdx.y][threadIdx.x]=a[row*width+i*tile+threadIdx.x];
        }
        else{
             Mds[threadIdx.y][threadIdx.x]=0;
        }
        if(col<r && (i*tile+threadIdx.y)<width){
            Nds[threadIdx.y][threadIdx.x]=b[(i*tile+threadIdx.y)*r+col];
        }
        else{
            Nds[threadIdx.y][threadIdx.x]=0;

        }
        __syncthreads();

        for(int k=0;k<tile;k++){
            p+=Mds[threadIdx.y][k]*Nds[k][threadIdx.x];
        }
        __syncthreads();

        }
    if(row<l && col<r){
            c[row*r+col]=p;
        }

}
int main(){
    int a[l*width],b[width*r],c[l*r],d[l*r];
    for(int i=0;i<l;i++){
        for(int j=0;j<width;j++){
            a[i*width+j]=i*width+j;
        }
    }
    for(int i=0;i<width;i++){
        for(int j=0;j<r;j++){
            b[i*r+j]=i*r+j;
        }

  }
    cudaEvent_t start,end;
    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);
    for(int i=0;i<l;i++){
        for(int j=0;j<r;j++){
            for(int k=0;k<width;k++){
                d[i*r+j]+=a[i*width+k]*b[k*r+j];
        }

    }}
    cudaEventRecord(end);
    cudaEventSynchronize(end);
    float time=0;
    cudaEventElapsedTime(&time,start,end);
    printf("CPU Time taken: %f ms\n",time);
    int *da,*db,*dc;
    cudaMalloc((void**)&da,l*width*sizeof(int));
    cudaMalloc((void**)&db,width*r*sizeof(int));
    cudaMalloc((void**)&dc,l*r*sizeof(int));
    cudaEvent_t start1,end1;
    cudaEventCreate(&start1);
    cudaEventCreate(&end1);
    cudaEventRecord(start1);
    cudaMemcpy(da,a,l*width*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(db,b,width*r*sizeof(int),cudaMemcpyHostToDevice);
    dim3 grid(ceil(r/2.0),ceil(l/2.0),1);
    dim3 block(2,2,1);
    kernel<<<grid,block>>>(da,db,dc);
    cudaMemcpy(c,dc,l*r*sizeof(int),cudaMemcpyDeviceToHost);
    cudaEventRecord(end1);
    cudaEventSynchronize(end1);
    float time1=0;
    cudaEventElapsedTime(&time1,start1,end1);
    printf("GPU Time taken: %f ms\n",time1);
    int count=0;
    for(int i=0;i<l;i++){
        for(int j=0;j<r;j++){
            if(c[i*r+j]!=d[i*r+j]){
                count++;
                printf("%d %d %d , %d\n", i,j,c[i*r+j],d[i*r+j]);
            }}}
    if(count==0){
        printf("Correct\n");
     }
    else{
        printf("failure\n");}
    cudaFree(da);
    cudaFree(db);
    cudaFree(dc);
    return 0;

}


CPU Time taken: 0.002720 ms
GPU Time taken: 0.158112 ms
Correct



In [ ]:
#MP3 3d convo

%%cuda -c "--gpu-architecture $gpu_arch"
#include <stdio.h>
#define l 16
#define width 5
#define rand1 4
#define outdim 4
#define indim 8
#define r 2
__constant__ float dfilter[width][width][width];
__global__ void kernel1(float *input,float *output){
    int row=blockIdx.y*blockDim.y+threadIdx.y;
    int col=blockIdx.x*blockDim.x+threadIdx.x;
    int z=blockIdx.z*blockDim.z+threadIdx.z;
    float p=0.0;
    for(int fz=0;fz<width;fz++){
        for(int fy=0;fy<width;fy++){
            for(int fx=0;fx<width;fx++){
                int inz=z-width/2+fz;
                int inx=col-width/2+fx;
                int iny=row-width/2+fy;
                if(inz>=0 && inx>=0 && iny>=0 && inz<l && inx<l && iny<l){
                    p=p+dfilter[fx][fy][fz]*input[inz*l*l+iny*l+inx];

                }

    }}}
    if(row>=0 && col>=0 && z>=0 && row<l && col<l && z<l){
    output[z*l*l+row*l+col]=p;}
}
__global__ void kernel2(float *input,float *output){
  int row=blockIdx.y*outdim+threadIdx.y-r;
  int col=blockIdx.x*outdim+threadIdx.x-r;
  int z=blockIdx.z*outdim+threadIdx.z-r;
  __shared__ float Ns[indim][indim][indim];
  if(row>=0 && col>=0 && z>=0 && row<l && col<l && z<l){
      Ns[threadIdx.z][threadIdx.y][threadIdx.x]=input[z*l*l+row*l+col];
  }
  else{
      Ns[threadIdx.z][threadIdx.y][threadIdx.x]=0;
  }
  __syncthreads();

  int tilerow=threadIdx.y-r;
  int tilecol=threadIdx.x-r;
  int tilez=threadIdx.z-r;
  if(row>=0 && z>=0 && col>=0 && row<l && col<l && z<l){
  if(tilerow>=0 && tilecol>=0 && tilez>=0 && tilerow<outdim && tilecol<outdim && tilez<outdim){
       float p=0.0;
       for(int fz=0;fz<width;fz++){
        for(int fy=0;fy<width;fy++){
            for(int fx=0;fx<width;fx++){
                p=p+dfilter[fx][fy][fz]*Ns[tilez+fz][tilerow+fy][tilecol+fx];

              }}}

       output[z*l*l+row*l+col]=p;
  }}
}

int main(){
    printf("start");
    float input[l*l*l];
    float filter[width][width][width];
    float output1[l*l*l];
    float output2[l*l*l];
    for(int i=0;i<l;i++){
        for(int j=0;j<l;j++){
            for(int k=0;k<l;k++){
                input[i*(l*l)+j*l+k]=10.0;
        }
  }}
    for(int i=0;i<width;i++){
        for(int j=0;j<width;j++){
            for(int k=0;k<width;k++){
                filter[i][j][k]=0.5;
        }
  }}
    printf("input done");
    float *dinput,*dfilter,*doutput1,*doutput2;
    cudaMalloc((void**)&dinput,l*l*l*sizeof(float));

    cudaMalloc((void**)&doutput1,l*l*l*sizeof(float));
    cudaMalloc((void**)&doutput2,l*l*l*sizeof(float));
    cudaMemcpy(dinput,input,l*l*l*sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(dfilter,filter,width*width*width*sizeof(float));
    dim3 grid(ceil(l/16.0),ceil(l/16.0),ceil(l/16.0));
    dim3 block(16,16,16);
    cudaEvent_t start1,end1;
    cudaEventCreate(&start1);
    cudaEventCreate(&end1);
    cudaEventRecord(start1);
    kernel1<<<grid,block>>>(dinput,doutput1);
    cudaMemcpy(output1,doutput1,l*l*l*sizeof(float),cudaMemcpyDeviceToHost);
    cudaEventRecord(end1);
    cudaEventSynchronize(end1);
    float time1=0;
    cudaEventElapsedTime(&time1,start1,end1);
    printf("GPU Time taken for naive : %f ms\n",time1);
    printf("naive done");
    dim3 grid1(ceil(l/8.0),ceil(l/8.0),ceil(l/8.0));
    dim3 block1(8,8,8);
    cudaEvent_t start2,end2;
    cudaEventCreate(&start2);
    cudaEventCreate(&end2);
    cudaEventRecord(start2);
    kernel2<<<grid1,block1>>>(dinput,doutput2);
    cudaMemcpy(output2,doutput2,l*l*l*sizeof(float),cudaMemcpyDeviceToHost);
    cudaEventRecord(end2);
    cudaEventSynchronize(end2);
    float time2=0;
    cudaEventElapsedTime(&time2,start2,end2);

    printf("GPU time taken for shared: %f ms\n",time2);
    int count=0;
    for(int i=0;i<l;i++){
        for(int j=0;j<l;j++){
            for(int k=0;k<l;k++){
                if(output1[i*l*l+j*l+k]!= output2[i*l*l+j*l+k]){
                    count++;

    }}}}

    if(count==0){
        printf("correct");
    }
    else{
        printf("incorrect");
    }
    cudaFree(dinput);
    cudaFree(dfilter);
    cudaFree(doutput1);





}

startinput doneGPU Time taken for naive : 0.303712 ms
naive doneGPU time taken for shared: 0.068160 ms
correct


In [ ]:
#MP4 sum reduction

%%cuda -c "--gpu-architecture $gpu_arch"
#define l 4090
#define bx 1024
#include <stdio.h>
__global__ void kernel(int *a,int *output){
    __shared__ int d_a[bx];
    int segment=2*blockIdx.x*blockDim.x;
    if((blockIdx.x*blockDim.x+threadIdx.x)<l){
        d_a[threadIdx.x]=a[segment+threadIdx.x]+a[segment+threadIdx.x+blockDim.x];
    }
    else{
        d_a[threadIdx.x]=0;
    }
    __syncthreads();
    for(int stride=(blockDim.x)/2;stride>=1;stride/=2){
        if(threadIdx.x<stride){
            d_a[threadIdx.x]+=d_a[threadIdx.x+stride];
        }
        __syncthreads();
    }
    if(threadIdx.x==0){
           atomicAdd(output,d_a[threadIdx.x])     ;
     }

}



int main(){
    int a[l];
    int output[1];
    for(int i=0;i<l;i++){
        a[i]=1;
    }
    int *da;
    int *doutput;
    cudaMalloc((void**)&da,l*sizeof(int));
    cudaMalloc((void**)&doutput,sizeof(int));
    cudaMemcpy(da,a,l*sizeof(int),cudaMemcpyHostToDevice);
    dim3 grid(ceil(l/2048.0),1,1);
    dim3 block(1024,1,1);
    kernel<<<grid,block>>>(da,doutput);
    cudaMemcpy(output,doutput,sizeof(int),cudaMemcpyDeviceToHost);
    printf("%d",output[0]);
    cudaFree(da);
    cudaFree(doutput);
    return 0;

}

4090


In [ ]:
#MP5
%%cuda -c "--gpu-architecture $gpu_arch"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda_runtime.h>

#define BLOCK_SIZE 1024
#define SECTION_SIZE 2048 // 2 * BLOCK_SIZE


__global__ void scan_kernel(float *input, float *output, float *global_sum, int n) {

    __shared__ float temp[SECTION_SIZE];

    int tx = threadIdx.x;
    int bx = blockIdx.x;


    int block_offset = bx * SECTION_SIZE;

    // Load 2 elements per thread
    int index1 = block_offset + (2 * tx);
    int index2 = block_offset + (2 * tx) + 1;


    if (index1 < n) temp[2 * tx]     = input[index1];
    else            temp[2 * tx]     = 0.0f;

    if (index2 < n) temp[2 * tx + 1] = input[index2];
    else            temp[2 * tx + 1] = 0.0f;

    __syncthreads();


    for (int stride = 1; stride <= BLOCK_SIZE; stride *= 2) {
        int index = (tx + 1) * stride * 2 - 1;
        if (index < SECTION_SIZE) {
            temp[index] += temp[index - stride];
        }
        __syncthreads();
    }


    if (tx == 0) {
        atomicAdd(global_sum, temp[SECTION_SIZE - 1]);
        temp[SECTION_SIZE - 1] = 0; // Clear for Exclusive Scan
    }
    __syncthreads();


    for (int stride = BLOCK_SIZE; stride >= 1; stride /= 2) {
        int index = (tx + 1) * stride * 2 - 1;
        if (index < SECTION_SIZE) {
            float t = temp[index - stride];
            temp[index - stride] = temp[index];
            temp[index] += t;
        }
        __syncthreads();
    }


    if (index1 < n) output[index1] = temp[2 * tx];
    if (index2 < n) output[index2] = temp[2 * tx + 1];
}


int main() {
    int num_elements = 4096; // 2 blocks * 2048
    size_t size = num_elements * sizeof(float);

    // Host Memory
    float *h_in = (float*)malloc(size);
    float *h_out = (float*)malloc(size);
    float h_global_sum_result = 0.0f;

    // Initialize Input (1.0f everywhere)
    for (int i = 0; i < num_elements; i++) {
        h_in[i] = 1.0f;
    }

    // Device Memory
    float *d_in, *d_out, *d_global_sum;
    cudaMalloc((void**)&d_in, size);
    cudaMalloc((void**)&d_out, size);
    cudaMalloc((void**)&d_global_sum, sizeof(float));

    // Initialize Global Sum to 0 on GPU
    cudaMemset(d_global_sum, 0, sizeof(float));
    cudaMemcpy(d_in, h_in, size, cudaMemcpyHostToDevice);

    // Launch Config
    dim3 dimGrid(2);
    dim3 dimBlock(BLOCK_SIZE);

    printf("Launching scan_kernel with atomicAdd...\n");

    // Timer
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    scan_kernel<<<dimGrid, dimBlock>>>(d_in, d_out, d_global_sum, num_elements);
    cudaEventRecord(stop);

    cudaDeviceSynchronize();

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("Kernel Time: %.4f ms\n", milliseconds);

    // Copy back
    cudaMemcpy(h_out, d_out, size, cudaMemcpyDeviceToHost);
    cudaMemcpy(&h_global_sum_result, d_global_sum, sizeof(float), cudaMemcpyDeviceToHost);



    // 1. Check Global Sum
    float cpu_total_sum = 0.0f;
    for(int i=0; i<num_elements; i++) cpu_total_sum += h_in[i];

    printf("Global Sum: GPU=%.2f, CPU=%.2f\n", h_global_sum_result, cpu_total_sum);

    // 2. Check Scan (Block Local Logic)
    // Block 0 should be 0..2047. Block 1 should also be 0..2047 (because it's local scan)
    bool scan_match = true;
    for (int i = 0; i < num_elements; i++) {
        // Expected value depends on block index
        float expected = (float)(i % 2048);
        if (fabs(h_out[i] - expected) > 0.01) {
            printf("Mismatch at %d: Expected %.1f, Got %.1f\n", i, expected, h_out[i]);
            scan_match = false;
            break;
        }
    }

    if (scan_match) printf("Scan Array: PASSED (Block-Local Logic)\n");
    else            printf("Scan Array: FAILED\n");

    // Cleanup
    cudaFree(d_in);
    cudaFree(d_out);
    cudaFree(d_global_sum);
    free(h_in);
    free(h_out);

    return 0;
}

Launching scan_kernel with atomicAdd...
Kernel Time: 0.1154 ms
Global Sum: GPU=4096.00, CPU=4096.00
Scan Array: PASSED (Block-Local Logic)



In [ ]:
#MP7
%%cuda -c "--gpu-architecture $gpu_arch"
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define ROWS 6
#define COLS 6

// --- JDS Matrix Structure ---
typedef struct {
    int num_rows;
    int num_cols;
    int max_row_len;    // Max non-zeros in any row (determines outer loop count)
    int *jd_ptr;        // Pointers to the start of each jagged diagonal
    int *col_index;     // Original column indices
    float *data;        // The non-zero values
    int *row_ptr;       // STORES ORIGINAL ROW INDICES (The permutation map)
} JDSMatrix;

// --- 1. CPU Reference (Dense Calculation) ---
// This represents the standard "Host" calculation for verification
void cpu_dense_mv(float matrix[ROWS][COLS], float *vector, float *out) {
    for (int i = 0; i < ROWS; i++) {
        float sum = 0.0f;
        for (int j = 0; j < COLS; j++) {
            sum += matrix[i][j] * vector[j];
        }
        out[i] = sum;
    }
}

// --- 2. GPU Simulation (JDS Kernel) ---
// This implements the JDS logic exactly as a GPU thread would execute it
void jds_spmv_kernel(JDSMatrix *mat, float *vector, float *out) {
    // Initialize output vector to 0
    for (int i = 0; i < mat->num_rows; i++) {
        out[i] = 0.0f;
    }

    // Iterate through each Jagged Diagonal (Column in the shifted matrix)
    for (int i = 0; i < mat->max_row_len; i++) {
        int start_index = mat->jd_ptr[i];
        int end_index   = mat->jd_ptr[i+1];

        // Process elements in this jagged diagonal
        for (int k = start_index; k < end_index; k++) {

            // Calculate which row (0 to N) inside the JDS block we are processing
            int row_in_jds_block = k - start_index;

            // Use row_ptr to map back to the ORIGINAL row index
            int original_row = mat->row_ptr[row_in_jds_block];

            // Perform the multiplication
            // out[original_row] += value * input_vector[column]
            out[original_row] += mat->data[k] * vector[mat->col_index[k]];
        }
    }
}

int main() {
    // --- SETUP HARDCODED DATA ---

    // 1. Input Vector (All 1.0)
    float vector[COLS] = {1.0, 1.0, 1.0, 1.0, 1.0, 1.0};

    // 2. Reference Dense Matrix (6x6)
    // Used only for the CPU check
    float dense_matrix[ROWS][COLS] = {
        {1, 0, 2, 0, 0, 0}, // Row 0 (2 nz)
        {0, 3, 0, 4, 0, 5}, // Row 1 (3 nz)
        {0, 0, 6, 0, 0, 0}, // Row 2 (1 nz)
        {7, 8, 9, 0, 0, 0}, // Row 3 (3 nz)
        {0, 0, 0, 0, 10,0}, // Row 4 (1 nz)
        {0, 0, 0, 0, 0, 11} // Row 5 (1 nz)
    };

    // 3. JDS Compressed Format Data
    // Sorted Rows: R1(len3), R3(len3), R0(len2), R2(1), R4(1), R5(1)

    float data[] = {
        3, 7, 1, 6, 10, 11,   // Jagged Diagonal 0 (Col 0 of shifted matrix)
        4, 8, 2,              // Jagged Diagonal 1 (Col 1 of shifted matrix)
        5, 9                  // Jagged Diagonal 2 (Col 2 of shifted matrix)
    };

    int col_index[] = {
        1, 0, 0, 2, 4, 5,     // Orig Col indices for JD 0
        3, 1, 2,              // Orig Col indices for JD 1
        5, 2                  // Orig Col indices for JD 2
    };

    int jd_ptr[] = {0, 6, 9, 11}; // Start indices of JDs

    // The Permutation Map (row_ptr)
    // Maps the sorted JDS rows back to: Row 1, Row 3, Row 0, Row 2, Row 4, Row 5
    int row_ptr[] = {1, 3, 0, 2, 4, 5};

    // Fill Struct
    JDSMatrix jds;
    jds.num_rows = ROWS;
    jds.num_cols = COLS;
    jds.max_row_len = 3;
    jds.data = data;
    jds.col_index = col_index;
    jds.jd_ptr = jd_ptr;
    jds.row_ptr = row_ptr; // Sending the permutation map to the kernel

    float cpu_result[ROWS];
    float gpu_result[ROWS];

    // --- EXECUTION (1 Iteration) ---

    // 1. Run CPU Reference
    cpu_dense_mv(dense_matrix, vector, cpu_result);

    // 2. Run GPU (JDS Kernel)
    jds_spmv_kernel(&jds, vector, gpu_result);



    int passed = 1;
    for (int i = 0; i < ROWS; i++) {
        // Compare values
        float diff = fabs(cpu_result[i] - gpu_result[i]);
        int match = (diff < 0.001); // Tolerance check
        if (!match) passed = 0;


    }


    if (passed) {
        printf("FINAL RESULT: PASSED (Outputs Match)\n");
    } else {
        printf("FINAL RESULT: FAILED (Outputs Do Not Match)\n");
    }

    return 0;
}

FINAL RESULT: PASSED (Outputs Match)

